In [1]:
import torch
import torch.nn as nn
import string
import random
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
filepath = "cleaned_text.txt"

In [2]:
def ensure_unicode(text):
    if isinstance(text, bytes):
        return text.decode('utf-8', 'ignore')
    elif isinstance(text, str):
        return text
    else:
        raise ValueError("Unsupported string type")

# Function to filter text
def filter_text(text, allowed_chars):
    return ''.join([char for char in text if char in allowed_chars])

In [11]:
with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
    content = file.read()
    content = ensure_unicode(content)
    all_chars = string.printable
    filtered_text = filter_text(content, all_chars)

# Use filtered text as 'all_text'
all_text = filtered_text
number_of_char = len(all_text)

all_chars = string.printable
number_of_chars = len(all_chars)
print(all_chars)
print(len(all_text))

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

55474549


In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out = self.embed(x)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell


In [21]:
class Generator:
    def __init__(self):
        self.chunk_len = 256 # puścić zwiększone do 350 dla lr=0.003 lr=0.0035 i lr=0.004
        self.num_epochs = 5000
        self.batch_size = 1
        self.print_every = 25
        self.hidden_size = 256
        self.num_layers = 3  # Nie ruszaj
        self.lr = 0.0006 # puścić to oraz dodatkowo bez ruszania chunk_len lr=0.004
        # czyli 5 puszczeń sieci :
        # 1: chunk_len=256 lr=0.0035
        # 2: chunk_len=256 lr=0.004
        # 3: chunk_len=350 lr=0.003
        # 4: chunk_len=350 lr=0.0035
        # 5: chunk_len=350 lr=0.004
        self.rnn = RNN(number_of_chars, self.hidden_size, self.num_layers, number_of_chars).to(device)

    def char_tensor(self, string):
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = all_chars.index(string[c])
        return tensor

    def get_random_batch(self):
        start_index = random.randint(0, len(all_text) - self.chunk_len)
        end_index = start_index + self.chunk_len + 1
        text_str = all_text[start_index:end_index]
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)

        for i in range(self.batch_size):
            text_input[i, :] = self.char_tensor(text_str[:-1])
            text_target[i, :] = self.char_tensor(text_str[1:])
        return text_input.long(), text_target.long()

    def generate(self, initial_string='T', prediction_length=100, temperature=0.85):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_string)
        predicted = initial_string

        for p in range(len(initial_string) - 1):
            _, (hidden, cell) = self.rnn(initial_input[p].view(1).to(device), hidden, cell)

        last_char = initial_input[-1]

        for p in range(prediction_length):
            output, (hidden, cell) = self.rnn(last_char.view(1).to(device), hidden, cell)
            output_dist = output.data.view(-1).div(temperature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            predicted_char = all_chars[top_char]
            predicted += predicted_char
            last_char = self.char_tensor(predicted_char)
        return predicted

    def train(self):
        optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        writer = SummaryWriter(f'runs/TextGenerationExperiment')
        print("=> starting training :)")

        start_time = time.time()
        for epoch in tqdm(range(1, self.num_epochs + 1), desc="Training", unit="epoch"):
            inp, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
            self.rnn.zero_grad()
            loss = 0
            inp = inp.to(device)
            target = target.to(device)

            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(inp[:, c], hidden, cell)
                loss += criterion(output, target[:, c])

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.rnn.parameters(), max_norm=1.0)
            optimizer.step()
            loss = loss.item() / self.chunk_len

            writer.add_scalar('Training Loss', loss, global_step=epoch)

            if epoch % self.print_every == 0:
                print(f'Loss: {loss}')
                print(self.generate())
                elapsed_time = time.time() - start_time
                estimated_total_time = elapsed_time / epoch * self.num_epochs
                print(f'Estimated total training time: {estimated_total_time // 60} minutes')

        # Save the model after training
        torch.save(self.rnn.state_dict(), 'trained_rnn.pth')
        print("=> training finished and model saved :)")

In [22]:
gennames = Generator()

In [25]:
gennames.train()

Model loaded.
=> starting training :)


Training:   0%|          | 0/5000 [00:00<?, ?epoch/s]


RuntimeError: cudnn RNN backward can only be called in training mode

In [27]:
def load_model(generator, model_path='trained_rnn.pth'):
    generator.rnn.load_state_dict(torch.load(model_path))
    generator.rnn.eval()
    print("Model loaded.")

# Load the model
load_model(gennames)

Model loaded.


In [30]:
def generate_text(generator, initial_string, length=100, temperature=0.6):
    generated_text = generator.generate(initial_string, length, temperature)
    return generated_text

In [32]:
benchmark_data = [
    ("Detective John entered the dimly lit room, only to find a pool of blood next to ", 50),
    ("The suspect had a solid alibi, but something in his eyes told the inspector that ", 100),
    ("She never expected the secret message hidden in the old locket would lead her to ", 60),
    ("As the clock struck midnight, the eerie silence was broken by the sound of ", 200),
    ("The missing pages from the diary hinted at a conspiracy that went all the way to ", 55),
    ("With every step in the abandoned warehouse, Mark could feel someone watching him from ", 45),
    ("The coded note left at the crime scene was the key to unraveling the mystery of ", 100),
    ("Just as she was about to give up, a mysterious figure emerged from the shadows and ", 60),
    ("The old detective had seen many cases, but nothing as chilling as the one involving ", 150),
    ("Every clue pointed to the butler, but the real mastermind behind the crimes was ", 50),
]

for prompt, max_new_tokens in benchmark_data:
    print(f"\"{generate_text(gennames, prompt, max_new_tokens)}\"\n")


"Detective John entered the dimly lit room, only to find a pool of blood next to the words, and his wife was not may not the procom"

"The suspect had a solid alibi, but something in his eyes told the inspector that he said to the fair sense of the fine she state of the particular as the carest with the man and app"

"She never expected the secret message hidden in the old locket would lead her to for the bottle men upon the same in the latter. And the ligh"

"As the clock struck midnight, the eerie silence was broken by the sound of him with the various and other so the laid to the restable
and have been has not discurriage with a many are the
declarable chart of
the police of the anger.
He repeated the clue in his latter side of"

"The missing pages from the diary hinted at a conspiracy that went all the way to action of the find the companions, with the bank of the"

"With every step in the abandoned warehouse, Mark could feel someone watching him from the charge are a goodness. In 